<a href="https://colab.research.google.com/github/jin-sj/pytorchzerotoall/blob/master/inception.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms

In [8]:
class SimpleModel(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(320, 10)

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = F.relu(self.mp(self.conv2(x)))
        x = x.view(in_size, -1)
        x = self.fc(x)
        return F.log_softmax(x, dim=1)


class InceptionA(torch.nn.Module):
    def __init__(self, input_channels):
        super(InceptionA, self).__init__()

        self.b1_conv2d1 = nn.Conv2d(input_channels, 24, kernel_size=1)

        self.b2_conv2d1 = nn.Conv2d(input_channels, 16, kernel_size=1)

        self.b3_conv2d1 = nn.Conv2d(input_channels, 16, kernel_size=1)
        self.b3_conv2d2 = nn.Conv2d(16, 24, kernel_size=5, padding=2)

        self.b4_conv2d1 = nn.Conv2d(input_channels, 16, kernel_size=1)
        self.b4_conv2d2 = nn.Conv2d(16, 24, kernel_size=3, padding=1)
        self.b4_conv2d3 = nn.Conv2d(24, 24, kernel_size=3, padding=1)


    def forward(self, x):
        x1 = F.avg_pool2d(x, kernel_size=3, stride=1, padding=1)
        x1 = self.b1_conv2d1(x1)

        x2 = self.b2_conv2d1(x)

        x3 = self.b3_conv2d1(x)
        x3 = self.b3_conv2d2(x3)

        x4 = self.b4_conv2d1(x)
        x4 = self.b4_conv2d2(x4)
        x4 = self.b4_conv2d3(x4)

        outputs = [x4, x2, x3, x1]
        return torch.cat(outputs, 1)


class InceptionNet(nn.Module):
    def __init__(self):
        super(InceptionNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(88, 20, kernel_size=5)

        self.incept1 = InceptionA(input_channels=10)
        self.incept2 = InceptionA(input_channels=20)

        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(1408, 10)

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = self.incept1(x)
        x = F.relu(self.mp(self.conv2(x)))
        x = self.incept2(x)
        x = x.view(in_size, -1)
        x = self.fc(x)
        return x
        #return F.log_softmax(x, dim=1)
        #return F.log_softmax(x, dim=1)



In [9]:
dev = "cpu"
if torch.cuda.is_available():
    dev = "cuda:0"
device = torch.device(dev)

BATCH_SIZE = 64
train_dataset = datasets.MNIST(root="./data/",
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)
test_dataset = datasets.MNIST(root="./data/",
                              train=False,
                              transform=transforms.ToTensor())
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=BATCH_SIZE,
                          shuffle=True)
test_loader = DataLoader(dataset=train_dataset,
                          batch_size=BATCH_SIZE,
                          shuffle=False)
model = InceptionNet().to(device)
#model = Net().to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
#optimizer = torch.optim.RMSprop(model.parameters(), lr=0.01)

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data.to(device)), Variable(target.to(device))
        output = model(data)

        loss = criterion(output, target)
        #loss = F.nll_loss(output, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} | Batch Status: {}/{} ({:.0f}%) | Loss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        with torch.no_grad():
            data, target = Variable(data.to(device)), Variable(target.to(device))
            output = model(data)

            test_loss += criterion(output, target).item()
            #test_loss += F.nll_loss(output, target).item()

            pred = torch.max(output.data, 1)[1] # 0th index is value, 1st index is class
            correct += pred.eq(target.data.view_as(pred)).sum()

    test_loss /= len(test_loader.dataset)
    print(f'===========================\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} '
          f'({100. * correct / len(test_loader.dataset):.0f}%)')

def main():
    epoch = 1
    for epoch_idx in range(epoch):
        train(epoch_idx)
    test()


In [10]:
main()

Train Epoch: 0 | Batch Status: 0/60000 (0%) | Loss: 2.295378
Train Epoch: 0 | Batch Status: 640/60000 (1%) | Loss: 2.318444
Train Epoch: 0 | Batch Status: 1280/60000 (2%) | Loss: 2.309789
Train Epoch: 0 | Batch Status: 1920/60000 (3%) | Loss: 2.295745
Train Epoch: 0 | Batch Status: 2560/60000 (4%) | Loss: 2.298311
Train Epoch: 0 | Batch Status: 3200/60000 (5%) | Loss: 2.296543
Train Epoch: 0 | Batch Status: 3840/60000 (6%) | Loss: 2.291620
Train Epoch: 0 | Batch Status: 4480/60000 (7%) | Loss: 2.288183
Train Epoch: 0 | Batch Status: 5120/60000 (9%) | Loss: 2.267980
Train Epoch: 0 | Batch Status: 5760/60000 (10%) | Loss: 2.284413
Train Epoch: 0 | Batch Status: 6400/60000 (11%) | Loss: 2.259992
Train Epoch: 0 | Batch Status: 7040/60000 (12%) | Loss: 2.258685
Train Epoch: 0 | Batch Status: 7680/60000 (13%) | Loss: 2.241180
Train Epoch: 0 | Batch Status: 8320/60000 (14%) | Loss: 2.179515
Train Epoch: 0 | Batch Status: 8960/60000 (15%) | Loss: 2.169746
Train Epoch: 0 | Batch Status: 9600/60